Idea - closing remarks of https://www.lesswrong.com/posts/FgjcHiWvADgsocE34/a-descriptive-not-prescriptive-overview-of-current-ai

a semantic search service that returns relevant literature

In [ ]:
# conda create aiproject python
# conda activate aiproject
# !pip install  anthropic
# https://github.com/anthropics/anthropic-cookbook/blob/main/misc/pdf_upload_summarization.ipynb

In [36]:
import json
file_path = 'alignment_texts.jsonl'

PUT API KEY

In [1]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
import os
anthropic = Anthropic(api_key= 'INSERT_API_KEY')

In [7]:
response = anthropic.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[{"role": "user", "content": "Hello, world"}]
)

In [8]:
# completion = anthropic.completions.create(    model="claude-3-haiku-20240307",    
#                                         max_tokens_to_sample=1024,    
#                                         prompt=f"{HUMAN_PROMPT} How do I?{AI_PROMPT}",)
# print(completion.completion)

1. Train Claude on https://github.com/moirage/alignment-research-dataset 
2. Prompt Claude to 
3. Ask Claude to create a markdown file with the top 10 clusters as topics and the top 10 papers in each cluster as subtopics.
4. Feed markdowns into quartz

In [9]:
response = anthropic.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[{"role": "user", "content": "How to train Claude on a dataset stored in jsonl?"}]
)

In [14]:
print(response)


Message(id='msg_01ETFj9cmQSqfcW3nh8CjNCW', content=[TextBlock(text="I don't actually have the ability to be trained on new datasets. I'm Claude, an AI assistant created by Anthropic to be helpful, harmless, and honest. I don't have access to external datasets or the capability to update my own knowledge or capabilities.", type='text')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=19, output_tokens=58))


In [48]:
MODEL_NAME="claude-3-5-sonnet-20240620"
def get_completion(client, prompt):
    return client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=[{
            "role": 'user', "content":  prompt
        }]
    ).content[0].text

In [43]:
def load_jsonl(file_path, num_lines=None):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if num_lines and len(data) >= num_lines:
                break
            item = json.loads(line.strip())
            if item.get('source_type') == 'latex':
                data.append(item)
    return data
    
def extract_relevant_fields(data):
    combined_entries = []
    for item in data:
        combined_entry = {
            'title': item.get('title'),
            'abstract': item.get('abstract'),
            # 'summary': item.get('summary')
        }
        combined_entries.append(combined_entry)
    return combined_entries

data = load_jsonl(file_path, num_lines=20)
combined_entries = extract_relevant_fields(data)


In [45]:
data

[{'source': 'arxiv',
  'source_type': 'latex',
  'converted_with': 'pandoc',
  'paper_version': '1806.09055v2',
  'title': 'DARTS: Differentiable Architecture Search',
  'authors': ['Hanxiao Liu', 'Karen Simonyan', 'Yiming Yang'],
  'date_published': '2018-06-24 00:06:13+00:00',
  'data_last_modified': '2019-04-23 06:29:32+00:00',
  'url': 'http://arxiv.org/abs/1806.09055v2',
  'abstract': 'This paper addresses the scalability challenge of architecture search by formulating the task in a differentiable manner. Unlike conventional approaches of applying evolution or reinforcement learning over a discrete and non-differentiable search space, our method is based on the continuous relaxation of the architecture representation, allowing efficient search of the architecture using gradient descent. Extensive experiments on CIFAR-10, ImageNet, Penn Treebank and WikiText-2 show that our algorithm excels in discovering high-performance convolutional architectures for image classification and rec

In [16]:
def generate_markdown(clusters, data):
    topic_dict = {}
    for idx, cluster in enumerate(clusters):
        if cluster not in topic_dict:
            topic_dict[cluster] = []
        topic_dict[cluster].append(data[idx])
    
    with open('output.md', 'w') as f:
        for topic, texts in topic_dict.items():
            f.write(f"## Topic {topic}\n")
            for text in texts:
                f.write(f"- {text}\n")
            f.write("\n")
# embeddings = get_embeddings(data)
# num_clusters = 5  # Adjust the number of clusters as needed
# clusters = cluster_data(embeddings, num_clusters)
# generate_markdown(clusters, data)

In [54]:
get_completion(anthropic, 
            f'Can you extract the major tags into a markdown format like Tag1: Paper1, Paper 2, Tag2: Paper 2, Paper 3 from the following papers? {combined_entries} - for example - mechanistic interpretability, red teaming, AI governance, value alignment, agent alignment - the papers could belong to more than 1 tag')

"Here's an attempt at extracting major tags in markdown format based on the papers provided:\n\nMechanistic Interpretability:\n- Can You Trust Your Model's Uncertainty? Evaluating Predictive Uncertainty Under Dataset Shift\n- Quantifying Perceptual Distortion of Adversarial Examples\n- Fooling the primate brain with minimal, targeted image manipulation\n\nRed Teaming:\n- Transfer of Adversarial Robustness Between Perturbation Types\n- Verifiably Safe Exploration for End-to-End Reinforcement Learning\n\nAI Governance:\n- Unsolved Problems in ML Safety\n- Predictability and Surprise in Large Generative Models\n- Accounting for the Neglected Dimensions of AI Progress\n\nValue Alignment:\n- An Efficient, Generalized Bellman Update For Cooperative Inverse Reinforcement Learning\n- Robot Planning with Mathematical Models of Human State and Action\n- Norms for Beneficial A.I.: A Computational Analysis of the Societal Value Alignment Problem\n\nAgent Alignment:\n- Learning What To Do by Simula

[{'title': 'Webinar with Congressman Ro Khanna: Challenges in IT Law and Governance', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://aipulse.org/feed/', 'value': 'Webinar with Congressman Ro Khanna: Challenges in IT Law and Governance'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://aipulse.org/webinar-with-congressman-ro-khanna-challenges-in-it-law-and-governance/'}], 'link': 'https://aipulse.org/webinar-with-congressman-ro-khanna-challenges-in-it-law-and-governance/', 'authors': [{'name': 'Laura Elbaum'}], 'author': 'Laura Elbaum', 'author_detail': {'name': 'Laura Elbaum'}, 'published': 'Mon, 22 Feb 2021 16:40:08 +0000', 'published_parsed': [2021, 2, 22, 16, 40, 8, 0, 53, 0], 'tags': [{'term': 'Webinars', 'scheme': None, 'label': None}], 'id': 'https://aipulse.org/?p=712', 'guidislink': False, 'summary': '<p>On Friday, February 19, the AI Pulse project hosted a web conversation on current issues in IT Governance with Congressman Ro Kh

\n\nMechanistic Interpretability:\n- Can You Trust Your Model's Uncertainty? Evaluating Predictive Uncertainty Under Dataset Shift\n- Quantifying Perceptual Distortion of Adversarial Examples\n- Fooling the primate brain with minimal, targeted image manipulation\n

\nRed Teaming:\n- Transfer of Adversarial Robustness Between Perturbation Types\n- Verifiably Safe Exploration for End-to-End Reinforcement Learning\n

\nAI Governance:\n- Unsolved Problems in ML Safety\n- Predictability and Surprise in Large Generative Models\n- Accounting for the Neglected Dimensions of AI Progress\n\nValue Alignment:\n- An Efficient, Generalized Bellman Update For Cooperative Inverse Reinforcement Learning\n- Robot Planning with Mathematical Models of Human State and Action\n- Norms for Beneficial A.I.: A Computational Analysis of the Societal Value Alignment Problem\n

\nAgent Alignment:\n- Learning What To Do by Simulating the Past\n- Reward Tampering Problems and Solutions in Reinforcement Learning: A Causal Influence Diagram Perspective\n- Making Efficient Use of Demonstrations to Solve Hard Exploration Problems"

Pain points
- Feed whole dataset
- Get exact markdown format